# Portfolio Optimization
This notebook demonstrates portfolio optimization techniques on portfolios of N assets. Bootstrapping will be used to estimate the standard errors of the allocation optimization functions. A simple example of two assets is first used to conceptualize the optimization techniques. Once demonstrated, portfolio optimization will be scalled to N assets. 